In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [2]:
# not that we only call lv_tet because we will tetrahedralize it, we are loading a hex mesh!
lv_tet = LV.from_pyvista_read("../sample_files/lvhexmesh.vtk") # load reference mesh (can also be done from other methods)
lv_tet.tetrahedralize(backend=TETRA_BACKEND.TETGEN, order=1, mindihedral=0.0, minratio=2.0) 
lv_tet.mesh

UnstructuredGrid,Information
N Cells,365443
N Points,95495
X Bounds,"-2.245e+01, 5.452e+01"
Y Bounds,"-4.001e+01, 4.336e+01"
Z Bounds,"-4.906e+01, 4.008e+01"
N Arrays,0


In [3]:
lv_tet.identify_surfaces(
  endo_epi_args=dict(threshold=90.0),
  apex_base_args=dict(ab_ql=0.04, ab_qh=0.69),
  aortic_mitral_args=dict(a1=0.4,
                          a2=0.5,
                          a3=0.3,
                          a4=75,
                          a5=130,
                          
                          m1=0.17,
                          m2=0.02,
                          m3=0.07,
                          m4=0.333
                          )
)
lv_tet.plot("surface", scalars=LV_MESH_DATA.SURFS_DETAILED.value,
        cmap="tab20_r")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(position=(49999.999999…

In [4]:
lv_tet.compute_fibers(surfRegionsIds=LV_FIBER_MODES.LDRB_1,
                    fiber_space="P_1",
                    alpha_endo_lv=60,  # Fiber angle on the endocardium
                    alpha_epi_lv=-60,  # Fiber angle on the epicardium
                    beta_endo_lv=0,  # Sheet angle on the endocardium
                    beta_epi_lv=0,  # Sheet angle on the epicardium
                    markers={},
                    ldrb_kwargs={},
                    save_xdmfs=False,
                    xdmfs_dir=None,
                    xdmfs_basename=None,
                    del_generated_files=True,
                    )

Calculating scalar fields
Compute scalar laplacian solutions with the markers: 
epi: 2
lv: 1
base: 22
  Num vertices: 95495
  Num cells: 365443
Solving linear variational problem.
  Apex coord: (45.03, -27.18, -39.29)
Solving linear variational problem.
Solving Laplace equation
 epi = 1, lv = 0
Solving linear variational problem.
 lv = 1, epi = 0
Solving linear variational problem.

Calculating gradients
Compute fiber-sheet system
Angles: 
alpha: 
 endo_lv: 60
 epi_lv: -60
 endo_septum: 60
 epi_septum: -60
 endo_rv: 60
 epi_rv: -60
beta: 
 endo_lv: 0
 epi_lv: 0
 endo_septum: 0
 epi_septum: 0
 endo_rv: 0
 epi_rv: 0


In [5]:
lv_tet.plot_streamlines(LV_FIBERS.F0, scalars=LV_FIBERS.F0_ANGLES, decimate_boundary=0.85)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [6]:
lv_hex = LV.from_pyvista_read("../sample_files/lvhexmesh.vtk") # reload hex mesh

In [7]:
lv_hex.regress_fibers(lv_tet)

array([[ 0.44340043, -0.03856583, -0.86033553, ...,  0.73821636,
         0.59019307,  0.32250374],
       [ 0.44918199, -0.0381963 , -0.86257769, ...,  0.74276369,
         0.58591022,  0.32135659],
       [ 0.45319723, -0.04095395, -0.85032745, ...,  0.75047274,
         0.5723072 ,  0.34024438],
       ...,
       [-0.22927665,  0.83820713,  0.52612674, ...,  0.99706292,
         0.13900393,  0.17470835],
       [-0.23288387,  0.87176569,  0.34581601, ...,  0.99182828,
         0.20993211,  0.20023326],
       [-0.20740527,  0.86580726,  0.39514304, ...,  0.99101153,
         0.18325412,  0.22068821]])

In [9]:
lv_hex.plot_streamlines(LV_FIBERS.F0, scalars=LV_FIBERS.F0_ANGLES, decimate_boundary=0.85)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…